# STEP 0: Use GPU

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Make sure you use the Colab GPU to run the testing phase

import torch
torch.cuda.is_available()

True

In [3]:
# Show the GPU info

!nvidia-smi

Thu Nov 25 10:17:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    29W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# STEP 1: Git clone your project

In [4]:

# Git clone your project
# git clone https://github.com/eritup45/VRDL_HW2_object_detection.git

%cd VRDL_HW2_object_detection/
!pwd
!ls

/content/VRDL_HW2_object_detection
/content/VRDL_HW2_object_detection
combine.py  hubconf.py	requirements.txt  train.py  w_exp2.zip
data	    models	runs		  utils
detect.py   README.md	test.py		  weights


# STEP 2: Install your requirements

In [5]:
# Install your requirments (torch, mmcv, ...)
# It is recommended that you wirte the requirements.txt file in your project.

# %cd mmdetection
# !pip install mmcv-full

# MAYBE try this
# !pip install -e .

!pip install googledrivedownloader
!pip install -r requirements.txt
# maybe install pytorch 1.6.0

In [6]:
# # Import your package and check the version

# import torch, torchvision
# print(torch.__version__, torch.cuda.is_available())

# import mmdet
# print(mmdet.__version__)

# from mmcv.ops import get_compiling_cuda_version, get_compiler_version
# print(get_compiling_cuda_version())
# print(get_compiler_version())

# from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# # You must import the below 5 packages 
# import os
# import cv2
# import time
# from tqdm import tqdm

# MY PACKAGES


from google_drive_downloader import GoogleDriveDownloader as gdd

import argparse
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, non_max_suppression,\
    apply_classifier, scale_coords, xyxy2xywh, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

import natsort
from tqdm import tqdm

# STEP 3: Wget testing data 

In [7]:
# 1. Download the testing data
# download data
gdd.download_file_from_google_drive(file_id='1lywCvkH-F2byfqillu_6lcQZSl3lHtph',
                          dest_path='./data/svhn.zip',
                          unzip=True)


# 2. Read the image_name and put them into the list
# You need to modify the path to fit your test_folder
# data_listdir = os.listdir("/content/mmdetection/test")


In [8]:
# 2. Download weights
gdd.download_file_from_google_drive(file_id='1df4qkbrfM_jXmei4JxRf-jy0CR4Kws0O',
                          dest_path='./w_exp2.zip',
                          unzip=True)

# STEP 4: Run inferene and bench mark

In [9]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

# # 1. Load your model and weights
# config = '/content/drive/MyDrive/NYCU/VRDL/HW2/VRDL_100_train_config.py'
# checkpoint = "/content/drive/MyDrive/NYCU/VRDL/HW2/demo.pth"

# # 2. Initialize the model
# model = init_detector(config, checkpoint, device='cuda:0')

# TODO: load source and weights correctly
source = 'data/svhn/test/'
weights = 'runs/train/exp2/weights/best.pt'
imgsz = 320

# Initialize
set_logging()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
half = device.type != 'cpu'  # half precision only supported on CUDA

# Load model
model = attempt_load(weights, map_location=device)  # load FP32 model
imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
if half:
    model.half()  # to FP16

dataset = LoadImages(source, img_size=imgsz)

img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
# run once
_ = model(img.half() if half else img) if device.type != 'cpu' else None

path_list, img_list = [], []
for path, img, im0s, vid_cap in dataset:
    print('')
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    path_list.append(path)
    img_list.append(img)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fusing layers... 


/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 316 layers, 21504999 parameters, 0 gradients, 51.4 GFLOPS


串流輸出內容已截斷至最後 5000 行。
image 8069/13068 /content/VRDL_HW2_object_detection/data/svhn/test/378933.png: 
image 8070/13068 /content/VRDL_HW2_object_detection/data/svhn/test/378963.png: 
image 8071/13068 /content/VRDL_HW2_object_detection/data/svhn/test/378970.png: 
image 8072/13068 /content/VRDL_HW2_object_detection/data/svhn/test/379056.png: 
image 8073/13068 /content/VRDL_HW2_object_detection/data/svhn/test/379079.png: 
image 8074/13068 /content/VRDL_HW2_object_detection/data/svhn/test/37908.png: 
image 8075/13068 /content/VRDL_HW2_object_detection/data/svhn/test/379097.png: 
image 8076/13068 /content/VRDL_HW2_object_detection/data/svhn/test/379103.png: 
image 8077/13068 /content/VRDL_HW2_object_detection/data/svhn/test/379131.png: 
image 8078/13068 /content/VRDL_HW2_object_detection/data/svhn/test/379156.png: 
image 8079/13068 /content/VRDL_HW2_object_detection/data/svhn/test/379174.png: 
image 8080/13068 /content/VRDL_HW2_object_detection/data/svhn/test/379188.png: 
image 8081/13068 /co

In [10]:
# Compare whether TA's data and my data is same.
data_listdir = sorted(path_list, key=lambda f: int(
    ''.join(filter(str.isdigit, f))))
    
# get index of sorted path
index = natsort.index_natsorted(path_list)
path_list = natsort.order_by_index(path_list, index)
img_list = natsort.order_by_index(img_list, index)

# print(data_listdir[:5])
# print(path_list[:5])
print(f'TA\'s and my Data EQUAL?: {data_listdir == path_list}')

TA's and my Data EQUAL?: True


### Please **screenshot** this cell, including the code and the output (your inference time), and put it into your report.

In [13]:
# # Test your inference time
# TEST_IMAGE_NUMBER = 100 # This number is fixed.
# test_img_list = []

# # Read image (Be careful with the image order)
# data_listdir.sort(key = lambda x: int(x[:-4]))
# for img_name in data_listdir[:TEST_IMAGE_NUMBER]:
#   img_path = os.path.join("/content/mmdetection/test", img_name)
#   img = cv2.imread(img_path)
#   test_img_list.append(img)

# start_time = time.time()
# for img in tqdm(test_img_list):
#     # your model prediction
#     pred = inference_detector(model, img)

# end_time  = time.time()
# print("\nInference time per image: ", (end_time - start_time) / len(test_img_list))

TEST_IMAGE_NUMBER = 100
test_img_list = img_list[:100]

start_time = time.time()
for img in tqdm(test_img_list):
    pred = model(img, augment=False)[0]

end_time = time.time()
# print(f'test_img_list[0].size(): {test_img_list[0].size()}')
print(f'\nTesting images length: {len(test_img_list)}')
print("\nInference time per image: ",
        (end_time - start_time) / len(test_img_list))

# Remember to screenshot!

100%|██████████| 100/100 [00:04<00:00, 20.54it/s]


Testing images length: 100

Inference time per image:  0.04876954793930054
